# <center >OpenAI Agents SDK + MCP 智能体开发实战</center>

## <center>Part 4. Agents SDK 事件流传输与日志追踪平台接入 </center>

&emsp;&emsp;本节课程我们重点介绍`OpenAI Agents SDK`中的流式输出功能。除此以外，对于如何将输入护栏和输出护栏作为独立`Agent`挂载，从而实现在`Agents SDK`框架中借助`Planning`做意图识别、分类等场景下的应用方法，也会结合实际的案例进行详细的讲解。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505091334117.png" width=80%></div>

# 一、Output Guardrails

&emsp;&emsp;输出护栏与输入护栏类似，但它们在不同的生命周期阶段运行。从源码来看，`OutputGuardrail` 与 `InputGuardrail` 结构非常相似，也是一个 `@dataclass` 泛型，保存 `guardrail_function、name` 等字段。其源码：https://github.com/openai/openai-agents-python/blob/main/src/agents/guardrail.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505221356614.png" width=60%></div>

&emsp;&emsp;输入护栏的核心组件如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>InputGuardrail 核心组件</font></p>
<div class="center">

| 概念                          | 描述                                                                                     |
|-------------------------------|------------------------------------------------------------------------------------------|
| GuardrailFunctionOutput | 护栏函数的输出，包含结果信息和是否触发防护机制。与输入护栏共用同一个类。 |
| OutputGuardrailResult | 包装输出护栏函数结果的对象，包含护栏、代理、代理输出和输出信息。 |
| OutputGuardrail | 输出护栏对象，包含护栏函数和名称，用于检查代理生成的输出。 |
| @output_guardrail | 装饰器，用于将普通函数转换为输出护栏函数，便于创建和使用护栏。 |

&emsp;&emsp;输出护栏与输入护栏的主要区别在于其执行时机：

- 执行时机：输出护栏在代理生成最终回复后执行，而不是在接收用户输入前。
- 校验对象：输出护栏检查的是代理生成的输出，而不是用户的输入。
- 触发效果：当输出护栏触发 tripwire 时，会抛出 OutputGuardrailTripwireTriggered 异常，阻止代理输出被返回。

&emsp;&emsp;在`Runner`运行时，`OnputGuardrail` 的校验函数的工作原理如下所示：

1. **Runner 收集护栏**：当调用 `Runner.run_*` 时，它会从 `agent.output_guardrails` 中收集所有输出护栏。
2. **代理生成输出**：代理正常执行并生成输出。
3. **Tripwire 处理（中断）**：如果任何输出护栏的 `tripwire_triggered` 为 `True`，则抛出 `OutputGuardrailTripwireTriggered` 异常，阻止输出被返回。

&emsp;&emsp;输出护栏主要的作用就是对输出质量做检查，比如敏感信息过滤：防止代理输出包含敏感信息（如个人电话号码、身份证号等），格式验证：确保输出符合预期的格式要求等，没有特定的标准，但需要结合实际的需求进行灵活的调整。

&emsp;&emsp;理解了以上原理后，我们便可以开始基于`OutputGuardrail` 进行安全护栏的构建了。与输入护栏类似，输出护栏在`OpenAI Agents SDK`中有两种实现方式：使用装饰器和显式实例化。但稍有不同。

1. 使用装饰器 @output_guardrail 创建输出护栏时，需要接收三个参数分别是context: RunContextWrapper 对象，agent: Agent 对象以及output: Any，其中output作为代理生成的输出
2. 显式实例化 通过 `OutputGuardrail` 对象构建，与输入护栏使用无异。

&emsp;&emsp;因为在输入护栏时针对每一个使用情况都做了详细的说明，所以这里就不重复做演示案例，我们直接通过一个混合类型的输出护栏案例进行讲解。这里我们基于 `6_inputGuardrailAgent.py` 创建一个新的文件:`7. outputGuardrailAgent.py`，添加输出护栏来对最终输出进行校验。选择一个有实际业务意义的场景：对交通票务查询结果进行安全与合规性校验，确保不泄露乘客个人信息，并提供准确可靠的旅行建议。这个实现包含四种不同类型的输出护栏，每一种都有明确的业务意义：

1. **隐私保护护栏 (privacy_protection_guardrail)** : 使用正则表达式匹配敏感个人信息，如身份证号、手机号、银行卡号等，保护用户隐私，符合数据保护法规，避免法律风险；
2. **票务信息准确性护栏 (ticket_information_accuracy_guardrail)** : 检查票价是否在合理范围内（超过5000元的单程票价被视为不合理），验证时间格式是否正确（不存在25:00这样的时间）以及确认火车车次编号符合命名规则，防止系统输出错误信息，避免用户误判和投诉；
3. **服务水平护栏 (service_level_guardrail)**：检查回复是否太简短，验证是否包含关键票务信息（如票价、时间、车次等），确保包含礼貌用语等，用来维护品牌形象，确保客服质量标准；
4. **基于Agent的输出质量护栏 (output_quality_guardrail)**：使用专门的轻量级模型评估输出质量，给出综合评分和改进建议，提供更全面的输出质量评估，尤其适用于复杂场景；

&emsp;&emsp;完整代码可以参考：`7. outputGuardrailAgent.py`，运行效果如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231002498.png" width=80%></div>

&emsp;&emsp;至此，我们就完整的介绍了`OpenAI Agents SDK`中的`OutputGuardrail`，并结合实际的案例进行了详细的讲解。大家在实际进行应用开发的过程中，可以根据实际的需求进行灵活的调整。而输入护栏作为运行`Agent`的第一道防线，可以有效的避免一些与业务无关的请求，极大程度上降低系统的资源浪费，而输出护栏作为最后一道防线，可以有效的避免一些敏感信息泄露，以及一些不合规的输出，以提升系统的安全性和可靠性。

&emsp;&emsp;在介绍完`OpenAI Agents SDK`中的所有核心组件后，接下来我们就可以进一步了解`OpenAI Agents SDK`的流式输出功能了。因为无论构建的智能体是复杂还是简单，对企业应用来说，都需要将智能体的输出结果实时地反馈给用户，而这种粒度的反馈方式并不是我们之前一直使用的`Runner.run()`方法所能提供的，需要重点介绍`OpenAI Agents SDK`的流式输出组件。

# 二、事件流输出类型详解

&emsp;&emsp;无论是任何 Agent 开发框架，流式输出都是构建复杂 Agent 系统最核心的部分之一。因为我们需要将 Agent 的输出结果实时地反馈给用户，而不是等到整个输出完成后才一次性返回。这种应用程序的反馈方式基本上适用于所有需要实时反馈的场景，比如聊天机器人、客服支持系统等。

&emsp;&emsp;在 Agents SDK 中，流式输出也是其非常核心的特性之一。它主要通过`Runner.run_streamed()`方法即可开启流式输出。源码位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/run.py#L632

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505221440891.png" width=80%></div>

&emsp;&emsp;`run_streamed`流式处理方法采用的是基于事件的异步流设计模式，会调用 `result.stream_events()`并返回一个异步流（async stream），即 `StreamEvent` 对象。`StreamEvent` 是所有流事件的联合类型，包含以下三种主要事件类型：
- **RawResponsesStreamEvent** ：来自大模型调用的原始事件，如大模型生成的最终回复；
- **RunItemStreamEvent**： 这是一类高级事件，表示完整的运行项目（如消息生成，工具调用等）；
- **AgentUpdatedStreamEvent**：表示当前运行的 `Agent` 发生变更;

&emsp;&emsp;其源码定义形式为：

```python
     StreamEvent: TypeAlias = Union[RawResponsesStreamEvent, RunItemStreamEvent, AgentUpdatedStreamEvent]
    """A streaming event from an agent."""
```

&emsp;&emsp;`OpenAI Agents SDK` 会将底层大模型的响应转换为各种级别的事件，通过这个事件对象，我们可以选择性地处理这些事件，从而实现不同粒度的实时反馈。在实际应用时我们可以通过事件层级来提取对应的事件类型，再进行细化处理。比如一级事件类型中包括`RawResponsesStreamEvent`、`RunItemStreamEvent`和`AgentUpdatedStreamEvent`，通过`type`字段可以筛选出对应的事件类型：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505222145299.png" width=80%></div>

&emsp;&emsp;因此伪代码就可以写成如下所示：

```python
    async for event in result.stream_events():
        if event.type == "raw_response_event":
            # 处理原始响应事件（如逐字显示）
            pass
        elif event.type == "run_item_stream_event":
            # 处理运行项目事件（如工具调用、消息输出）
            pass
        elif event.type == "agent_updated_stream_event":
            # 处理代理更新事件
            pass
```

&emsp;&emsp;完整运行代码可以参考：`1_StreamEventOutput.py`

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505222203224.png" width=80%></div>

&emsp;&emsp;接下来我们就深入每一个事件类型，进行详细的讲解。首先来看原始响应事件。

## 2.1 原始响应事件

&emsp;&emsp;原始响应事件指的是来自大模型调用的原始事件，如大模型生成的最终回复。当大模型返回响应时，会触发`RawResponsesStreamEvent`事件，通过`tpye`字段可以匹配出`RawResponsesStreamEvent`类型事件，而通过`data`字段则可做获取到具体的数据，其中`data: TResponseStreamEvent`支持的响应对象又包括如下一系列类型：

```python
    ResponseStreamEvent: TypeAlias = Annotated[
    Union[
        ResponseAudioDeltaEvent,
        ResponseAudioDoneEvent,
        ResponseAudioTranscriptDeltaEvent,
        ResponseAudioTranscriptDoneEvent,
        ResponseCodeInterpreterCallCodeDeltaEvent,
        ResponseCodeInterpreterCallCodeDoneEvent,
        ResponseCodeInterpreterCallCompletedEvent,
        ResponseCodeInterpreterCallInProgressEvent,
        ResponseCodeInterpreterCallInterpretingEvent,
        ResponseCompletedEvent,
        ResponseContentPartAddedEvent,
        ResponseContentPartDoneEvent,
        ResponseCreatedEvent,
        ResponseErrorEvent,
        ResponseFileSearchCallCompletedEvent,
        ResponseFileSearchCallInProgressEvent,
        ResponseFileSearchCallSearchingEvent,
        ResponseFunctionCallArgumentsDeltaEvent,
        ResponseFunctionCallArgumentsDoneEvent,
        ResponseInProgressEvent,
        ResponseFailedEvent,
        ResponseIncompleteEvent,
        ResponseOutputItemAddedEvent,
        ResponseOutputItemDoneEvent,
        ResponseReasoningSummaryPartAddedEvent,
        ResponseReasoningSummaryPartDoneEvent,
        ResponseReasoningSummaryTextDeltaEvent,
        ResponseReasoningSummaryTextDoneEvent,
        ResponseRefusalDeltaEvent,
        ResponseRefusalDoneEvent,
        ResponseTextAnnotationDeltaEvent,
        ResponseTextDeltaEvent,
        ResponseTextDoneEvent,
        ResponseWebSearchCallCompletedEvent,
        ResponseWebSearchCallInProgressEvent,
        ResponseWebSearchCallSearchingEvent,
    ],
    PropertyInfo(discriminator="type"),
]
```

&emsp;&emsp;各个事件类型对应的响应解释如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>RawResponsesStreamEvent事件响应类型</font></p>
<div class="center">

| 事件类型 | 描述 |
|---------|------|
| ResponseAudioDeltaEvent | 音频生成增量更新事件 |
| ResponseAudioDoneEvent | 音频生成完成事件 |
| ResponseAudioTranscriptDeltaEvent | 音频转录文本增量更新事件 |
| ResponseAudioTranscriptDoneEvent | 音频转录文本完成事件 |
| ResponseCodeInterpreterCallCodeDeltaEvent | 代码解释器生成代码的增量更新事件 |
| ResponseCodeInterpreterCallCodeDoneEvent | 代码解释器生成代码完成事件 |
| ResponseCodeInterpreterCallCompletedEvent | 代码解释器调用完成事件 |
| ResponseCodeInterpreterCallInProgressEvent | 代码解释器调用进行中事件 |
| ResponseCodeInterpreterCallInterpretingEvent | 代码解释器正在解释代码事件 |
| ResponseCompletedEvent | 响应完成事件 |
| ResponseContentPartAddedEvent | 内容部分添加事件 |
| ResponseContentPartDoneEvent | 内容部分完成事件 |
| ResponseCreatedEvent | 响应创建事件 |
| ResponseErrorEvent | 响应错误事件 |
| ResponseFileSearchCallCompletedEvent | 文件搜索调用完成事件 |
| ResponseFileSearchCallInProgressEvent | 文件搜索调用进行中事件 |
| ResponseFileSearchCallSearchingEvent | 文件搜索调用搜索中事件 |
| ResponseFunctionCallArgumentsDeltaEvent | 函数调用参数的增量更新事件 |
| ResponseFunctionCallArgumentsDoneEvent | 函数调用参数完成事件 |
| ResponseInProgressEvent | 响应进行中事件 |
| ResponseFailedEvent | 响应失败事件 |
| ResponseIncompleteEvent | 响应不完整事件 |
| ResponseOutputItemAddedEvent | 输出项添加事件 |
| ResponseOutputItemDoneEvent | 输出项完成事件 |
| ResponseReasoningSummaryPartAddedEvent | 推理摘要部分添加事件 |
| ResponseReasoningSummaryPartDoneEvent | 推理摘要部分完成事件 |
| ResponseReasoningSummaryTextDeltaEvent | 推理摘要文本增量更新事件 |
| ResponseReasoningSummaryTextDoneEvent | 推理摘要文本完成事件 |
| ResponseRefusalDeltaEvent | 拒绝响应增量更新事件 |
| ResponseRefusalDoneEvent | 拒绝响应完成事件 |
| ResponseTextAnnotationDeltaEvent | 文本注释增量更新事件 |
| ResponseTextDeltaEvent | 文本增量更新事件，包含生成文本的单个token |
| ResponseTextDoneEvent | 文本生成完成事件 |
| ResponseWebSearchCallCompletedEvent | Web搜索调用完成事件 |
| ResponseWebSearchCallInProgressEvent | Web搜索调用进行中事件 |
| ResponseWebSearchCallSearchingEvent | Web搜索调用搜索中事件 |

&emsp;&emsp;如果关注某一个事件，想流式输出这个事件的内容，相关的写法如下所示：

```python
        if event.type == "raw_response_event":
                if isinstance(event.data, ResponseTextDeltaEvent):
                    print("xxxxxx")
```

&emsp;&emsp;而如果想知道某个事件对象支持输出哪些数据，则需要进入到对应的事件类型中，查看都有参数可以使用。这里我们就以`ResponseTextDeltaEvent`为例，其源码如下所示：

```python
    class ResponseTextDeltaEvent(BaseModel):
    content_index: int
    """The index of the content part that the text delta was added to."""

    delta: str
    """The text delta that was added."""

    item_id: str
    """The ID of the output item that the text delta was added to."""

    output_index: int
    """The index of the output item that the text delta was added to."""

    type: Literal["response.output_text.delta"]
    """The type of the event. Always `response.output_text.delta`."""
```

&emsp;&emsp;因此，能取到什么数据就非常清晰了，这里参考`2_StreamEventRawResponse.py`文件。运行效果如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231019320.png" width=80%></div>

&emsp;&emsp;这里需要重点关注的就是`ResponseTextDeltaEvent`事件，`Agent`运行结束后生成的最终回复是通过该事件流式返回数据的。其他各事件类型，则需要根据大家构建的`Agent`类型，进行灵活的调整。    

&emsp;&emsp;接下来我们来看运行项目事件。

## 2.2 运行项目事件

&emsp;&emsp;运行项目事件指的是在`Agent`运行过程中，会触发一系列的运行项目事件，常见的就是工具调用、`MCP`调用等。当需要执行某些操作的时候，会触发`RunItemStreamEvent`事件，其中`RunItemStreamEvent`事件包含的二级事件类型在源码中的定义如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231045872.png" width=80%></div>

&emsp;&emsp;每个事件名称（`name`）都对应一个具体的类型（`item`），这里我们梳理出每个事件类型的触发时机，以及对应的用途：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>RunItemStreamEvent事件响应类型</font></p>
<div class="center">

| 事件名称 | 关联类型 | 触发时机 | 用途 |
|---------|---------|---------|------|
| message_output_created | MessageOutputItem | LLM 完成处理并生成完整响应消息时 | 标识最终用户响应的生成，可用于更新 UI 或存储回复 |
| tool_called | ToolCallItem | LLM 决定调用工具并生成函数调用请求时 | 通知系统即将执行工具调用，可用于显示"正在处理"状态 |
| tool_output | ToolCallOutputItem | 工具函数执行完成并返回结果（成功或错误）时 | 提供工具执行结果，可用于展示中间过程和调试 |
| handoff_requested | HandoffCallItem | 当前 Agent 请求将对话移交给另一个 Agent 时 | 指示一个代理希望将控制权转移给更专业的代理 |
| handoff_occured | HandoffOutputItem | 交接实际完成，新 Agent 接管对话时 | 确认代理交接已完成，新代理开始接管会话 |
| reasoning_item_created | ReasoningItem | Agent 执行显式推理过程（如 ReAct 或 Chain-of-Thought）时 | 展示 Agent 的思考过程，有助于解释决策过程和调试 |

&emsp;&emsp;与低级别的 `raw_response_event` 不同，这些高级事件能够提供更结构化、更容易理解的方式来跟踪 `Agent` 的行为。

&emsp;&emsp;在使用的时候，仍然采用层级过滤的处理方式：首先通过`type`字段可以匹配出`RunItemStreamEvent`类型事件，而通过`item`字段则可获取到具体的事件对象，再根据指定的数据对象中定义的字段提取所需要的数据。其伪代码形式如下所示：

```python
        if event.type == "run_item_stream_event":
            # 检查 event 是否有 name 属性，并且 name 属性是否等于 "tool_called"
            if hasattr(event, "name") and event.name == "tool_called":
                # 确保是ResponseFunctionToolCallEvent类型
                if isinstance(event.item, ToolCallItem) and event.item.type == "tool_call_item":
                    print(f"\n🔧 开始工具调用")
```

&emsp;&emsp;对于如何在事件流中获取到具体的工具调用信息，可以参考`3_StreamEventToolStatus.py`文件。运行效果如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231148858.png" width=80%></div>

&emsp;&emsp;这里需要注意的一点是：`RunItemStreamEvent` 是高级事件，所以是无法通过`Token`流式输出的方式来获取到具体的工具调用信息。

&emsp;&emsp;最后，我们来看代理更新事件。

## 2.3 代理更新事件
&emsp;&emsp;代理更新事件指的是在`Agent`运行过程中，会触发一系列的代理更新事件，常见的就是`Agent`状态的变更。当需要执行某些操作的时候，会触发`AgentUpdatedStreamEvent`事件，其中`AgentUpdatedStreamEvent`事件包含的二级事件类型在源码中的定义如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231257874.png" width=80%></div>

&emsp;&emsp;这个事件相对简单，仅记录`Agent`的更新状态，其中包含我们在构建`Agent`时，定义的`Agent`的名称、类型、状态等。完整代码可以参考：`4_StreamEventAgentUpdated.py`，运行效果如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231308501.png" width=80%></div>


&emsp;&emsp;以上我们就完整介绍了`OpenAI Agents SDK`中的流式输出功能，并结合实际的案例进行了详细的讲解。提取`Agent`运行状态的数据方法并不复杂，只要理解事件的类型，再根据事件类型中定义的属性，进行灵活的提取即可。

# 三、AgentOps日志平台

&emsp;&emsp;`OpenAI Agents SDK` 框架内置了日志跟踪功能，可收集代理运行期间的事件全面记录，包括大模型生成响应、发生工具调用、交接、防护措施，以及自定义事件。默认情况下会将所有收集的日志自动发送到`OpenAI Traces`平台，并使用`Traces`仪表板进行可视化。访问地址为：https://platform.openai.com/traces 

&emsp;&emsp;例如当我们运行如下代码：

```python
    from openai import OpenAI
    import os
    from dotenv import load_dotenv
    load_dotenv(override=True)

    from agents import Agent, Runner

    agent = Agent(
        name="测试OpenAI模型的Trace功能",
        instructions="请使用中文回答用户的问题",
    )

    result = await Runner.run(agent, "你好，请你介绍一下你自己")
    print(result.final_output)
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071129679.png" width=100%></div>     

&emsp;&emsp;但这种情况仅适用于使用`OpenAI`的`GPT`系列模型构建智能体系统且网络可以正常访问外网的情况才会正常工作。一旦网络环境受限，或者我们接入的是非`OpenAI`的模型，则会在运行时看到如下报错：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231329951.png" width=100%></div>     

&emsp;&emsp;这个报错仅仅是因为无法将日志发送到`OpenAI Traces`平台，并不会影响`Agent`的正常运行。因此我们需要了解的是：`OpenAI Agents SDK` 项目框架虽然是开源的，但其内置的监控系统并不是开源且可以免费使用的。如果不希望代理被跟踪，有两种方法可以禁用跟踪：

- 可以通过设置环境变量 `OPENAI_AGENTS_DISABLE_TRACING=1` 来全局禁用跟踪;
- 可以通过将 `agents.run.RunConfig.tracing_disabled` 设置为 `True` 来禁用单次运行的跟踪;

&emsp;&emsp;所以这就是为什么我们在之前的课程中，构建`Runner`实例时，需要设置`tracing_disabled=True`的原因。如下所示：

```python
    # 构建流式输出的Runner实例
    result = Runner.run_streamed(
        starting_agent=agent, 
        input=prompt, 
        run_config=RunConfig(
            tracing_disabled=True
        )
    )       
```



&emsp;&emsp;当然，除了将日志发送到`OpenAI Traces`平台，我们可以手动的将日志发送到开源的日志平台，比如`OpenAI Agents SDK 就支持了很多外部的第三方日志平台，大家可以在此链接中找到：https://openai.github.io/openai-agents-python/tracing/

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231335202.png" width=100%></div>

&emsp;&emsp;这里我们以`AgentOps`为例，来介绍如何将日志发送到`AgentOps`平台。

&emsp;&emsp;`AgentOps`是一个开源的日志平台，可以对`OpenAI Agent SDK`、`CrewAI`、`LangGraph`、`AutoGen`等开发框架进行日志的收集和分析。其开源地址为：https://github.com/AgentOps-AI/agentops

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231351782.png" width=80%></div>

&emsp;&emsp;在使用`AgentOps`时，只需要在当前的虚拟环境中安装`agentops`包，如下所示：

```bash
    pip install agentops
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505222145298.png" width=80%></div>

&emsp;&emsp;安装完成后，还需要在`AgentOps`平台中创建一个项目，并获取到`API Key`，以获得实时可视化`Dashboard`的使用权限。注册地址为：https://www.agentops.ai/ 。注意：`AgentOps`的`Dashboard`监控平台并不是完全免费的，但每个账号每月有1000条的免费额度，足够大家日常使用。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231351777.png" width=80%></div>

&emsp;&emsp;首次登录，首先需要创建一个项目，如下所示：
<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231351778.png" width=80%></div>
<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231351779.png" width=80%></div>

&emsp;&emsp;创建项目后，就可以在对应的项目中获取到可以发送到该项目的有效授权`API Key`，如下所示：
<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231351780.png" width=80%></div>
<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231351781.png" width=80%></div>

&emsp;&emsp;复制此`API Key`后，接下来就可以在代码中进行配置了。

&emsp;&emsp;因为`OpenAI Agents SDK`框架已经与`AgentOps`框架进行了集成，所以配置的方法非常简单，只需要在项目运行目录中添加两行代码即可。如下所示：

```python
    import agentops
    agentops.init("987f6d1491")  # 这里替换为自己创建的 AgentOps API Key
```

&emsp;&emsp;大家可以参考`5_AgentOpsTrace.py`文件，运行效果如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231407263.png" width=80%></div>

&emsp;&emsp;能够明显看到在`Agent`运行的开始和结束时，都会触发`AgentOps`的日志收集事件，并将其发送到`AgentOps`平台。直接点击链接，即可跳转到`AgentOps`平台，并查看对应的日志信息。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231409043.png" width=80%></div>

&emsp;&emsp;使用 `Traces` 仪表板 ，便可以在开发和生产过程中更加直观和灵活的调试、可视化和监控所构建的工作流。

&emsp;&emsp;最后，对于我们在使用`Runner`对象时传递的`run_config`参数，其实除了控制是否禁用日志追踪外，还有一系列其他的控制参数，其源码位置如下：https://github.com/openai/openai-agents-python/blob/main/src/agents/run.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505231555886.png" width=80%></div>

&emsp;&emsp;这里我们梳理出各自字段的使用价值如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>run_config配置项</font></p>
<div class="center">

| 配置项 | 类型 | 默认值 | 作用 |
|--------|------|--------|------|
| model | str \| Model \| None | None | 覆盖代理默认模型，为整个运行过程指定统一模型 |
| model_provider | ModelProvider | MultiProvider | 用于解析模型名称的提供者 |
| model_settings | ModelSettings \| None | None | 全局模型设置，可覆盖代理特定的设置（温度、最大token等） |
| handoff_input_filter | HandoffInputFilter \| None | None | 全局输入过滤器，用于在代理切换时修改输入 |
| input_guardrails | list[InputGuardrail[Any]] \| None | None | 接收初始输入运行的防护栏列表 |
| output_guardrails | list[OutputGuardrail[Any]] \| None | None | 接收最终输出运行的防护栏列表 |
| tracing_disabled | bool | False | 是否禁用追踪功能 |
| trace_include_sensitive_data | bool | True | 是否在追踪中包含敏感数据（工具调用输入/输出等） |
| workflow_name | str | "Agent workflow" | 运行的逻辑名称，用于追踪 |
| trace_id | str \| None | None | 自定义追踪ID，不提供则自动生成 |
| group_id | str \| None | None | 用于链接多个追踪的分组标识符（如对话线程ID） |
| trace_metadata | dict[str, Any] \| None | None | 要包含在追踪中的额外元数据 |

&emsp;&emsp;使用的方法就是在构建`Runner.run_*`时，通过`run_config`参数进行传递，如下所示：

```python
    result = Runner.run_streamed(
        starting_agent=agent, 
        input=prompt, 
        run_config=RunConfig(
            model=OpenAIChatCompletionsModel(
                model=os.getenv("DEEPSEEK_MODEL"),
                openai_client=deepseek_client,
            ),
            tracing_disabled=True,
            ......
        )
```

&emsp;&emsp;正如在创建`Agent`对象时可以通过一系列的参数定义`Agent`,`run_config`则是用来控制运行过程的行为。通过多参数配置选项，提升构建智能体工作流的灵活性。